# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6507159d7c1374ba58c8f6d3'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [24]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "address": "123 Main Street, Greenwich",
    "postcode": "SE10 8RT",
    "rating": "Not Rated"  # Since it hasn't been rated yet
}

In [25]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)


In [26]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print("New restaurant 'Penang Flavours' has been added to the database.")
else:
    print("Failed to add the new restaurant.")

New restaurant 'Penang Flavours' has been added to the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [27]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query ={'BusinessType':'Restaurant/Cafe/Canteen'}
fields =['BusinessTypeID', 'BusinessType']

establishments.find_one(query, fields)

{'_id': ObjectId('6507159d7c1374ba58c8f6d3'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [34]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"name": "Penang Flavours"},
    {'$set':
         {'BusinessTypeID': 1}
    }
)

In [35]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({'name': 'Penang Flavours'})
if updated_restaurant:
    pprint(updated_restaurant)
else:
    print("Restaurant not found or update failed.")

{'BusinessTypeID': 1,
 '_id': ObjectId('6508fb6fa036a28222647084'),
 'address': '123 Main Street, Greenwich',
 'name': 'Penang Flavours',
 'postcode': 'SE10 8RT',
 'rating': 'Not Rated'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
count_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {count_dover}")


Number of documents with LocalAuthorityName 'Dover': 994


In [37]:
# Delete all documents where LocalAuthorityName is "Dover"
deleted_count = establishments.delete_many({"LocalAuthorityName": "Dover"}).deleted_count
print(f"Deleted {deleted_count} documents with LocalAuthorityName 'Dover'")


Deleted 994 documents with LocalAuthorityName 'Dover'


In [38]:
# Check if any remaining documents include Dover
remaining_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})
if remaining_dover == 0:
    print("All 'Dover' documents have been deleted.")
else:
    print(f"There are still {remaining_dover} documents with LocalAuthorityName 'Dover'.")


All 'Dover' documents have been deleted.


In [39]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()
if sample_document:
    pprint(sample_document)
else:
    print("No documents found in the collection.")

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6507159d7c1374ba58c8f9b5'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [40]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [
    {'$set': {'Latitude': {'$toDouble': '$Latitude'}}},  # Convert Latitude to double
    {'$set': {'Longitude': {'$toDouble': '$Longitude'}}}  # Convert Longitude to double
])


Use `update_many` to convert `RatingValue` to integer numbers.

In [41]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [42]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({"RatingValue": {"$ne": None}}, [
    {'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}  # Convert RatingValue to integer
])

In [43]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()
if sample_document:
    pprint(sample_document)
else:
    print("No document found in the collection.")

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'Latitude': None,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'Longitude': None,
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6507159d7c1374ba58c8f9b5'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'me